In [1]:
import pandas as pd
import pickle
from shapely.geometry import Polygon
from shapely.geometry import MultiPoint #AMBOS HACEN LO MISMO
from scipy.spatial import ConvexHull  #AMBOS HACEN LO MISMO

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
all_plays = pd.read_csv("../processed_data/weeks_frame1/frame1_process/all_players_play.csv",index_col=0)
all_plays.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
0,2018090600:75:1,91.73,26.67,0.00,0.01,0.02,289.57,240.93,310.0,Matt Ryan,2.0,QB,away,left
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left
2,2018090600:75:1,91.35,44.16,0.02,0.03,0.01,290.45,16.86,2495454.0,Julio Jones,11.0,WR,away,left
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left
4,2018090600:75:1,90.78,36.15,0.00,0.00,0.00,257.61,193.97,2533040.0,Mohamed Sanu,12.0,WR,away,left


In [4]:
defenses = pd.read_csv("../processed_data/processing_defenses/defense_players_plays_no_rz.csv",index_col=0)
defenses.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
1,2018090600:75:1,88.89,36.47,0.01,0.01,0.01,105.63,66.66,79848.0,Malcolm Jenkins,27.0,SS,home,left
3,2018090600:75:1,86.31,22.01,0.09,0.42,0.01,70.12,168.91,2495613.0,Corey Graham,24.0,FS,home,left
5,2018090600:75:1,73.64,28.70,0.03,0.02,0.01,103.05,219.41,2534832.0,Rodney McLeod,23.0,FS,home,left
7,2018090600:75:1,86.48,31.12,0.09,0.03,0.04,95.90,33.36,2552315.0,Jordan Hicks,58.0,MLB,home,left
8,2018090600:75:1,82.67,20.53,0.03,0.07,0.01,81.14,174.57,2552689.0,Ronald Darby,21.0,CB,home,left


In [5]:
offenses = pd.read_csv("../processed_data/processing_defenses/offense_players_plays_no_rz.csv",index_col=0)
offenses.head()

,id,x,y,s,a,dis,o,dir,nflId,displayName,jerseyNumber,position,team,playDirection
0,2018090600:75:1,91.73,26.67,0.00,0.01,0.02,289.57,240.93,310.0,Matt Ryan,2.0,QB,away,left
2,2018090600:75:1,91.35,44.16,0.02,0.03,0.01,290.45,16.86,2495454.0,Julio Jones,11.0,WR,away,left
4,2018090600:75:1,90.78,36.15,0.00,0.00,0.00,257.61,193.97,2533040.0,Mohamed Sanu,12.0,WR,away,left
6,2018090600:75:1,98.25,26.63,0.01,0.01,0.00,265.88,194.51,2543583.0,Devonta Freeman,24.0,RB,away,left
10,2018090600:75:1,91.29,21.83,0.01,0.02,0.01,250.34,151.83,2555415.0,Austin Hooper,81.0,TE,away,left


In [6]:
plays = pd.read_csv("../processed_data/processing_defenses/plays.csv",index_col=0)
plays.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
4,2018090600:256:1,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False,True


In [7]:
plays_no_rz = plays[plays["redZone"]==False]
plays_no_rz.head()

,id,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,redZone
0,2018090600:75:1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,False
1,2018090600:146:1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,False
2,2018090600:168:1,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,False
3,2018090600:190:1,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,False
5,2018090600:320:1,(10:10) (Shotgun) N.Foles pass short left to N...,1,2,8,PHI,play_type_pass,PHI,4,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,4.0,"3 DL, 3 LB, 5 DB",0.0,0.0,10:10:00,14.0,NaN,NaN,C,4,4,-0.344096,False,False


In [8]:
players = pd.read_csv("../processed_data/clean/players.csv",index_col=0)
players.head()

,height,weight,birthDate,collegeName,position,displayName
nflId,,,,,,
2539334,1.83,86,1990-09-10,Washington,CB,Desmond Trufant
2539653,1.78,84,1988-11-01,Southeastern Louisiana,CB,Robert Alford
2543850,1.75,84,1991-12-18,Purdue,SS,Ricardo Allen
2555162,1.85,103,1994-11-04,Louisiana State,MLB,Deion Jones
2555255,1.91,105,1993-07-01,Minnesota,OLB,De'Vondre Campbell


Vamos a empezar identificando cuantos jugadores hay en la zona profunda. Esta zona está 10 o 15 yardas más lejos de la línea de scrimmage.

In [9]:
transf_def = pd.DataFrame(columns = ["id"])

In [10]:
transf_def["id"] = plays_no_rz["id"]

In [11]:
def deepzonePlayers(id):
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x > deepzone:
            number_players+=1
        elif playDirection == "left" and player_x < deepzone:
            number_players+=1
    return number_players

In [12]:
def defensivelineZone(id): # desde la y del balón y 7 yardas para arriba y abajo
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            number_players+=1
            #print(row["displayName"])
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            number_players+=1
            #print(row["displayName"])
    return number_players

In [13]:
def hookzonePlayers(id): # dentro de 23.6 y 29.7 (dar un poco de margen, 2 yardas por lado)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_hook = -1
    
    if playDirection == "right":
        x_hook = los + 15
    elif playDirection == "left":
        x_hook = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x > los + 1.5 :
                number_players+=1
            elif yFootball + 7 >= 31.7 and not( player_x <= los + 1.5 and player_y <= 31.7 and player_y >= yFootball -7):
                number_players+=1
            elif yFootball - 7 <= 21.6 and not( player_x <= los + 1.5 and player_y >= 21.6 and player_y <= yFootball +7):
                number_players+=1
        elif playDirection == "left" and player_x >= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x < los - 1.5 :
                number_players+=1
            elif yFootball + 7 >= 31.7 and not(player_x >= los - 1.5 and player_y <= 31.7 and player_y >= yFootball - 7):
                number_players+=1
            elif yFootball - 7 <= 21.6 and not(player_x >= los - 1.5 and player_y >= 21.6 and player_y <= yFootball + 7):
                number_players+=1
    return number_players

Vamos a asignar a la zona curl como la zona que está entre las hash marks y los números y la flat la zona fuera de los números

In [14]:
def curlzonePlayers(id): # está entre la zona hook y los números (12 yd hasta 21.6 yd y 31.7 yd hasta 41.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==id)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_curl = -1
    
    if playDirection == "right":
        x_curl = los + 15
    elif playDirection == "left":
        x_curl = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and ((player_x <= los +1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x <= los + 1.5 and player_y < 21.6 and player_y >= yFootball -7)):
                continue
                #number_players+=1
                #print(1,row["displayName"])
            elif yFootball + 7 > 31.7 and not(player_x <= los + 1.5 and player_y > 31.7 and player_y <= yFootball + 7):
                number_players+=1
                #print(2,row["displayName"])
            elif yFootball - 7 < 21.6 and not(player_x <= los + 1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                #print(3,row["displayName"])
        elif playDirection == "left" and player_x >= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball + 7 > 31.7 and yFootball - 7 < 21.6 and ((player_x >= los - 1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x >= los -1.5 and player_y < 21.6 and player_y >= yFootball - 7)):
                continue
                #number_players+=1
                #print(4,row["displayName"])
            elif yFootball + 7 > 31.7 and not(player_x >= los -1.5 and player_y > 31.7 and player_y <= yFootball +7):
                number_players+=1
                #print(5,row["displayName"])
            elif yFootball - 7 < 21.6 and not(player_x >= los -1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                #print(6,row["displayName"])
    return number_players

In [15]:
def flatzonePlayers(id): # está fuera de los números (0 yd hasta 12 yd y 41.3 yd hasta 53.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==id]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==id]
    playDirection = defenses[defenses["id"]==id]["playDirection"].values[0]
    x_flat = -1
    
    if playDirection == "right":
        x_flat = los + 15
    elif playDirection == "left":
        x_flat = los - 15
    
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
        elif playDirection == "left" and player_x >= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
    return number_players

In [16]:
def numberCoverPlayers(playId):
    defenders = defenses[defenses["id"]==playId]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        else:
            number_players += 1
    
    return number_players

In [17]:
def checkNumberIsGood(dl,dp,hz,cz,fz,total):
    return dl + dp + hz +cz +fz == total

In [18]:
transf_def["defensivelinezonePlayers"] = transf_def["id"].apply(defensivelineZone)

In [19]:
transf_def["deepzonePlayers"] = transf_def["id"].apply(deepzonePlayers)

In [20]:
transf_def["hookzonePlayers"] = transf_def["id"].apply(hookzonePlayers)

In [21]:
transf_def["curlzonePlayers"] = transf_def["id"].apply(curlzonePlayers)

In [22]:
transf_def["flatzonePlayers"] = transf_def["id"].apply(flatzonePlayers)

In [23]:
transf_def["numberCoverPlayers"] = transf_def["id"].apply(numberCoverPlayers)

In [24]:
transf_def["checkNumberIsGood"] = transf_def.apply(lambda x: checkNumberIsGood(x["defensivelinezonePlayers"],x["deepzonePlayers"],x["hookzonePlayers"],x["curlzonePlayers"],x["flatzonePlayers"],x["numberCoverPlayers"]),axis=1)

In [25]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood
0,2018090600:75:1,0,1,3,2,1,7,True
1,2018090600:146:1,0,1,3,2,1,7,True
2,2018090600:168:1,0,1,2,2,2,7,True
3,2018090600:190:1,1,0,2,3,1,7,True
5,2018090600:320:1,1,0,3,3,1,8,True
...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True
19113,2018123015:3932:17,0,0,1,4,2,7,True
19114,2018123015:3969:17,0,1,1,4,1,7,True
19115,2018123015:4057:17,0,0,2,3,2,7,True


In [27]:
def calculateArea(playId):
    defenders = defenses[defenses["id"]==playId]
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    
    return hull.volume

In [28]:
transf_def["defenseArea"] = transf_def["id"].apply(calculateArea)

In [29]:
def calculateAreaNoLine(playId):
    defenders = defenses[defenses["id"]==playId]
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        player_x = row["x"]
        player_y = row["y"]
        
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue

        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    ch_area = MultiPoint(position_players).convex_hull.area
    
    return hull.volume

In [30]:
transf_def["defenseAreaCoverDefenders"] = transf_def["id"].apply(calculateAreaNoLine)

In [31]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020
...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860


In [32]:
def calculateWidth(playId):
    defenders = defenses[defenses["id"]==playId]
    
    players_y = []
    
    for index,row in defenders.iterrows():
        players_y.append(row["y"])
        
    return max(players_y) - min(players_y)

In [33]:
transf_def["width"] = transf_def["id"].apply(calculateWidth)

In [34]:
def calculateHeight(playId):
    defenders = defenses[defenses["id"]==playId]
    
    players_x = []
    
    for index,row in defenders.iterrows():
        players_x.append(row["x"])
        
    return abs(max(players_x) - min(players_x))

In [35]:
transf_def["height"] = transf_def["id"].apply(calculateHeight)

In [36]:
def calculatePlayDirection(id):
    return defenses[defenses["id"]==id]["playDirection"].values[0]

In [37]:
transf_def["playDirection"] = transf_def["id"].apply(calculatePlayDirection)

Number of offensive players

In [40]:
def calculateNumberQBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    qbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="QB")]
    
    return len(qbs)

In [41]:
transf_def["numberQBs"] = transf_def["id"].apply(calculateNumberQBs)

In [42]:
def calculateNumberWRs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    wrs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="WR")]
    
    return len(wrs)

In [43]:
transf_def["numberWRs"] = transf_def["id"].apply(calculateNumberWRs)

In [44]:
def calculateNumberTEs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    tes = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"]=="TE")]
    
    return len(tes)

In [45]:
transf_def["numberTEs"] = transf_def["id"].apply(calculateNumberTEs)

In [46]:
def calculateNumberRBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    rbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"].isin(["HB","RB"]))]
    
    return len(rbs)

In [47]:
transf_def["numberRBs"] = transf_def["id"].apply(calculateNumberRBs)

In [48]:
def calculateNumberFBs(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    fbs = offensive_players[(offensive_players["id"]==playId)&(offensive_players["position"] == "FB")]
    
    return len(fbs)

In [49]:
transf_def["numberFBs"] = transf_def["id"].apply(calculateNumberFBs)

In [11]:
def calculateNumberAnotherPosition(playId):
    offensive_players = offenses[offenses["id"]==playId]
    
    another_positions = offensive_players[(offensive_players["id"]==playId)&(~offensive_players["position"].isin(["QB","FB","RB","HB","TE","WR"]))]
    df.insert(0, 'newColMean', df.mean(1))
    return len(another_positions)

In [14]:
transf_def.insert(18, 'numberOffensivePlayersAnotherPosition', transf_def["id"].apply(calculateNumberAnotherPosition))

Jugadores defensivos

In [52]:
def calculateNumberSafeties(playId):
    defense = defenses[defenses["id"]==playId]
    
    safeties = defenses[(defenses["id"]==playId)&(defenses["position"].isin(["SS","FS","S"]))]
    
    return len(safeties)

In [53]:
transf_def["numberSafeties"] = transf_def["id"].apply(calculateNumberSafeties)

In [54]:
def calculateNumberLBs(playId):
    defense = defenses[defenses["id"]==playId]
    
    lbs = defenses[(defenses["id"]==playId)&(defenses["position"].isin(["LB","OLB","MLB","ILB"]))]
    
    return len(lbs)

In [55]:
transf_def["numberLBs"] = transf_def["id"].apply(calculateNumberLBs)

In [56]:
def calculateNumberCBs(playId):
    defense = defenses[defenses["id"]==playId]
    
    cbs = defenses[(defenses["id"]==playId)&(defenses["position"].isin(["CB","DB"]))]
    
    return len(cbs)

In [59]:
transf_def["numberCBs"] = transf_def["id"].apply(calculateNumberCBs)

In [60]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberSafeties,numberLBs,numberCBs
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,3,2,2
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,2,2,3
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,2,2,3
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,3,1,3
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,2,3,2
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,2,2,3
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,2,1,4
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,3,2,2


Ahora calculamos cual el es el lado fuerte

In [10]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberSafeties,numberLBs,numberCBs
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,3,2,2
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,2,2,3
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,2,2,3
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,3,1,3
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,2,3,2
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,2,2,3
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,2,1,4
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,3,2,2


In [22]:
for play in transf_def[(transf_def["numberQBs"]==2)]["id"]:
    yFootball = all_plays[(all_plays["id"]==play)&(all_plays["displayName"]=="Football")]["y"].values[0]
    
    qbs = all_plays[(all_plays["id"]==play)&(all_plays["position"]=="QB")]
    counter = 0
    for index,row in qbs.iterrows():
        if abs(float(row["y"]) - float(yFootball)) > 3:
            counter += 1
    
    if counter != 1:
        print(play)

2018093012:3195:4
2018111809:658:11


Estas dos jugadas son las únicas en las que uno de los dos qbs actúa como RB

In [9]:
def calculateStrongSide(playId,numberTEs,numberFBs,numberRBs,numberQBs,playDirection):
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    
    strongSide = ""
    
    if numberTEs == 0: #hay que terminar esto
        if numberQBs == 1:
            players = offenses[offenses["id"]==playId]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if row["position"] != "QB" and row["y"] < yFootball:
                    number_players_0tof += 1
                elif row["position"] != "QB" and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
                
        elif numberQBs == 2:
            players = offenses[offenses["id"]==playId]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if (row["displayName"]!= "Teddy Bridgewater" and row["displayName"]!= "Drew Brees") and row["y"] < yFootball:
                    number_players_0tof += 1
                elif (row["displayName"]!= "Teddy Bridgewater" and row["displayName"]!= "Drew Brees") and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
    elif numberTEs == 1:
        te = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[0]
        if (playDirection == "left" and te > yFootball) or (playDirection == "right" and te < yFootball):
            strongSide = "right"
        elif (playDirection == "right" and te > yFootball) or (playDirection == "left" and te < yFootball):
            strongSide = "left"
    elif numberTEs == 2:
        te1 = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[0]
        te2 = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[1]
        
        if (te1 > yFootball and te2 > yFootball) or (te1 < yFootball and te2 < yFootball):
            sameSide = True
        elif (te1 > yFootball and te2 < yFootball) or (te1 < yFootball and te2 > yFootball):
            sameSide = False
        
        if (sameSide and playDirection == "left") or (sameSide and playDirection == "right"):
            strongSide = "right"
        elif (sameSide and playDirection == "right") or (sameSide and playDirection == "left"):
            strongSide = "left"
        elif not(sameSide) and numberFBs == 1:
            fb = offenses[(offenses["id"]==playId)&(offenses["position"]=="FB")]["y"].values[0]
            if (playDirection == "left" and fb < yFootball) or (playDirection == "right" and fb >yFootball):
                strongSide = "left"
            elif (playDirection == "left" and fb > yFootball) or (playDirection == "right" and fb < yFootball):
                strongSide = "right"
                
        elif not(sameSide) and numberFBs == 0 and numberRBs == 1:
            rb = offenses[(offenses["id"]==playId)&(offenses["position"].isin(["RB","HB"]))]["y"].values[0]
            
            if (playDirection == "left" and rb < yFootball) or (playDirection == "right" and rb >yFootball):
                strongSide = "left"
            elif (playDirection == "left" and rb > yFootball) or (playDirection == "right" and rb < yFootball):
                strongSide = "right"
                
            if rb == yFootball:
                wr1 = offenses[(offenses["id"]==playId)&(offenses["position"]=="WR")]["y"].values[0]
                wr2 = offenses[(offenses["id"]==playId)&(offenses["position"]=="WR")]["y"].values[1]
                
                
                if (wr1 > yFootball and wr2 > yFootball) or (wr1 < yFootball and wr2 < yFootball):
                    sameSideWrs = True
                elif (wr1 > yFootball and wr2 < yFootball) or (wr1 < yFootball and wr2 > yFootball):
                    sameSideWrs = False
                    
                if (sameSideWrs and playDirection == "left") or (sameSideWrs and playDirection == "right"):
                    strongSide = "right"
                elif (sameSideWrs and playDirection == "right") or (sameSideWrs and playDirection == "left"):
                    strongSide = "left"
                elif not(sameSideWrs):
                    strongSide = "right"
        elif not(sameSide) and numberFBs == 0 and numberRBs > 1:
            players = offenses[offenses["id"]==playId]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if row["position"] != "QB" and row["y"] < yFootball:
                    number_players_0tof += 1
                elif row["position"] != "QB" and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
        elif not(sameSide) and numberFBs == 0 and numberFBs == 0:
            players = offenses[offenses["id"]==playId]
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for index,row in players.iterrows():
                if row["position"] != "QB" and row["y"] < yFootball:
                    number_players_0tof += 1
                elif row["position"] != "QB" and row["y"] > yFootball:
                    number_players_fto53 += 1
            
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof) or number_players_fto53 == number_players_0tof:
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
    elif numberTEs == 3:
        te1 = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[0]
        te2 = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[1]
        te3 = offenses[(offenses["id"]==playId)&(offenses["position"]=="TE")]["y"].values[2]
        
        tes = [te1,te2,te3]
        
        if (te1 > yFootball and te2 > yFootball and te3 > yFootball) or (te1 < yFootball and te2 < yFootball and te3 < yFootball):
            sameSide = True
        else:
            sameSide = False
            
        if (sameSide and playDirection == "left") or (sameSide and playDirection == "right"):
            strongSide = "right"
        elif (sameSide and playDirection == "right") or (sameSide and playDirection == "left"):
            strongSide = "left"
        elif not(sameSide):
            number_players_fto53 = 0 #number of players from football to the line 53.3
            number_players_0tof = 0 #number of players from line 0 to football
            for te in tes:
                if te > yFootball:
                    number_players_fto53 += 1
                elif te < yFootball:
                    number_players_0tof +=1
            if (playDirection == "left" and number_players_fto53 > number_players_0tof) or (playDirection == "right" and number_players_fto53 < number_players_0tof):
                strongSide = "right"
            elif (playDirection == "left" and number_players_fto53 < number_players_0tof) or (playDirection == "right" and number_players_fto53 > number_players_0tof):
                strongSide = "left"
            
    elif numberTEs == 4:
        strongSide = "right"
    
    
    
        
    return strongSide

In [11]:
transf_def["strongSide"] = transf_def.apply(lambda x: calculateStrongSide(x["id"],x["numberTEs"],x["numberFBs"],x["numberRBs"],x["numberQBs"],x["playDirection"]),axis=1)

In [16]:
def calculateNumberPlayersDefenseSS(idPlay,strongSide,playDirection):
    yFootball = all_plays[(all_plays["id"]==idPlay)&(all_plays["displayName"]=="Football")]["y"].values[0]
    defenders = defenses[defenses["id"]==idPlay]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        
        if (playDirection == "left" and strongSide == "left" and row["y"] >= 0 and row["y"] <= yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= yFootball and row["y"] <= 53.3) or (playDirection == "right" and strongSide == "left" and row["y"] <= 53.3 and row["y"]>= yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]>=0 and row["y"]<=yFootball):
            number_players += 1
    
    return number_players

In [17]:
transf_def["numberPlayersDefenseStrongSide"] = transf_def.apply(lambda x: calculateNumberPlayersDefenseSS(x["id"],x["strongSide"],x["playDirection"]),axis=1)

In [18]:
def calculateNumberPlayersDefenseWS(idPlay,strongSide,playDirection):
    yFootball = all_plays[(all_plays["id"]==idPlay)&(all_plays["displayName"]=="Football")]["y"].values[0]
    defenders = defenses[defenses["id"]==idPlay]
    
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        
        if (playDirection == "left" and strongSide == "left" and row["y"] <= 53.3 and row["y"] > yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= 0 and row["y"] < yFootball) or (playDirection == "right" and strongSide == "left" and row["y"] >= 0 and row["y"]< yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]> yFootball and row["y"]<=53.3):
            number_players += 1
    
    return number_players

In [19]:
transf_def["numberPlayersDefenseWeakSide"] = transf_def.apply(lambda x: calculateNumberPlayersDefenseWS(x["id"],x["strongSide"],x["playDirection"]),axis=1)

In [20]:
def calculateNumberPlayersOffenseSS(playId,strongSide,playDirection,numberQBs):
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    offensive_players = offenses[offenses["id"]==playId]
    
    number_players = 0
    for index,row in offensive_players.iterrows():
        if numberQBs == 1:
            if row["position"] != "QB" and ((playDirection == "left" and strongSide == "left" and row["y"] >= 0 and row["y"] <= yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= yFootball and row["y"] <= 53.3) or (playDirection == "right" and strongSide == "left" and row["y"] <= 53.3 and row["y"]>= yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]>=0 and row["y"]<=yFootball)):
                number_players += 1
                #print(row["displayName"])
        elif numberQBs == 2:
            if row["position"] == "QB" and abs(float(row["y"]) - float(yFootball)) < 3 and (playId != "2018093012:3195:4" and playId != "2018111809:658:11"):
                continue
            elif row["position"] == "QB" and playId == "2018093012:3195:4" and row["displayName"]=="Joe Flacco":
                continue
            elif row["position"] == "QB" and playId == "2018111809:658:11" and row["displayName"]=="Drew Brees":
                continue
            elif (playDirection == "left" and strongSide == "left" and row["y"] >= 0 and row["y"] <= yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= yFootball and row["y"] <= 53.3) or (playDirection == "right" and strongSide == "left" and row["y"] <= 53.3 and row["y"]>= yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]>=0 and row["y"]<=yFootball):
                number_players += 1
            
    return number_players

In [21]:
transf_def["numberPlayersOffenseStrongSide"] = transf_def.apply(lambda x: calculateNumberPlayersOffenseSS(x["id"],x["strongSide"],x["playDirection"],x["numberQBs"]),axis=1)

In [22]:
def calculateNumberPlayersOffenseWS(playId,strongSide,playDirection,numberQBs):
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    offensive_players = offenses[offenses["id"]==playId]
    
    number_players = 0
    for index,row in offensive_players.iterrows():
        if numberQBs == 1:
            if row["position"] != "QB" and ((playDirection == "left" and strongSide == "left" and row["y"] <= 53.3 and row["y"] > yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= 0 and row["y"] < yFootball) or (playDirection == "right" and strongSide == "left" and row["y"] >= 0 and row["y"]< yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]> yFootball and row["y"]<=53.3)):
                number_players += 1
                #print(row["displayName"],row["position"])
        elif numberQBs == 2:
            if row["position"] == "QB" and abs(float(row["y"]) - float(yFootball)) < 3 and (playId != "2018093012:3195:4" and playId != "2018111809:658:11"):
                continue
            elif row["position"] == "QB" and playId == "2018093012:3195:4" and row["displayName"]=="Joe Flacco":
                continue
            elif row["position"] == "QB" and playId == "2018111809:658:11" and row["displayName"]=="Drew Brees":
                continue
            elif (playDirection == "left" and strongSide == "left" and row["y"] <= 53.3 and row["y"] > yFootball) or (playDirection == "left" and strongSide == "right" and row["y"] >= 0 and row["y"] < yFootball) or (playDirection == "right" and strongSide == "left" and row["y"] >= 0 and row["y"]<= yFootball) or (playDirection == "right" and strongSide == "right" and row["y"]> yFootball and row["y"]<=53.3):
                number_players += 1

    return number_players

In [23]:
transf_def["numberPlayersOffenseWeakSide"] = transf_def.apply(lambda x: calculateNumberPlayersOffenseWS(x["id"],x["strongSide"],x["playDirection"],x["numberQBs"]),axis=1)

In [24]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,3,2,2,left,3,4,3,2,5,True,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034,3.376032,0.036025,0.036025,6.974268,6.974268
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,2,2,3,right,3,4,2,3,5,True,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455,3.178611,0.034284,0.034284,6.975511,6.975511
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,2,2,3,left,5,2,4,1,5,True,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745,2.337434,0.025211,0.025211,6.985594,6.985594
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,3,1,3,right,2,5,2,3,5,True,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455,3.192871,0.037004,0.036351,6.973569,6.969707
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,2,3,3,left,3,5,3,2,5,True,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279,4.281979,0.042752,0.045692,7.978624,7.980418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,2,3,2,right,3,4,3,2,5,True,-3.91,0,-2,0.000000,0.017582,0.019208,0.000000,4.867507,0.041344,0.048035,6.970468,6.959971
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,2,2,3,right,3,4,3,2,5,True,-0.97,0,-2,0.000000,0.016636,0.018979,0.019819,2.120069,0.021384,0.021384,6.987781,6.987781
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,2,1,4,left,3,4,2,3,5,True,0.07,-1,-1,0.018878,0.016636,0.019862,0.019789,2.307450,0.023858,0.023858,6.982958,6.982958
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,3,2,2,right,3,4,2,3,5,True,-0.94,-1,-1,0.000000,0.019526,0.019136,0.020001,2.948817,0.031275,0.031275,6.982128,6.982128


In [94]:
def calculateDifferenceWidth(playId,defenseWidth):
    offense_players = offenses[offenses["id"]==playId]
    
    players_y = []
    
    for index,row in offense_players.iterrows():
        players_y.append(row["y"])
        
    return max(players_y) - min(players_y) - defenseWidth

In [95]:
transf_def["differenceOffenseVsDefenseWidth"] = transf_def.apply(lambda x: calculateDifferenceWidth(x["id"],x["width"]),axis=1)

In [ ]:
def calculateDifferenceStrongSides(playId,numberOffenseStrong,numberDefenseStrong):
    return numberOffenseStrong - numberDefenseStrong

In [ ]:
transf_def["differenceOffenseVsDefenseStrongSide"] = transf_def.apply(lambda x: calculateDifferenceStrongSides(x["id"],x["numberPlayersOffenseStrongSide"],x["numberPlayersDefenseStrongSide"]),axis=1)

In [ ]:
def calculateDifferenceWeakSides(playId,numberOffenseWeak,numberDefenseWeak):
    return numberOffenseWeak - numberDefenseWeak

In [ ]:
transf_def["differenceOffenseVsDefenseWeakSide"] = transf_def.apply(lambda x: calculateDifferenceWeakSides(x["id"],x["numberPlayersOffenseWeakSide"],x["numberPlayersDefenseWeakSide"]),axis=1)

Ahora a calcular el tamaño/peso por zonas

In [139]:
def calculateHeightByWeightDeep(playId):
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    deepzone = -1
    
    if playDirection == "right":
        deepzone = los + 15
    elif playDirection == "left":
        deepzone = los - 15
    
    value = 0
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]: #muy dificil que pase, pero por si acaso
            continue
        player_x = row["x"]
        if playDirection == "right" and player_x > deepzone:
            number_players+=1
            player_height = players.loc[row["nflId"]]["height"]
            player_weight = players.loc[row["nflId"]]["weight"]
            value += player_height/player_weight 
        elif playDirection == "left" and player_x < deepzone:
            number_players+=1
            player_height = players.loc[row["nflId"]]["height"]
            player_weight = players.loc[row["nflId"]]["weight"]
            value += player_height/player_weight 
            
    weight_by_height = 0
    if number_players != 0:
        weight_by_height=value/number_players
    else:
        weight_by_height = 0
            
    return weight_by_height

In [140]:
transf_def["HeightByWeightDeep"] = transf_def["id"].apply(calculateHeightByWeightDeep)

In [143]:
def calculateHeightByWeightHook(playId):
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_hook = -1
    
    if playDirection == "right":
        x_hook = los + 15
    elif playDirection == "left":
        x_hook = los - 15
    
    value = 0
    number_players = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x > los + 1.5 :
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight 
            elif yFootball + 7 >= 31.7 and not( player_x < los + 1.5 and player_y <= 31.7 and player_y >= yFootball -7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight 
            elif yFootball - 7 <= 21.6 and not( player_x < los + 1.5 and player_y >= 21.6 and player_y <= yFootball +7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight 
        elif playDirection == "left" and player_x >= x_hook and (player_y >= 21.6 and player_y <= 31.7):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and player_x < los - 1.5 :
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight 
            elif yFootball + 7 >= 31.7 and not(player_x > los - 1.5 and player_y <= 31.7 and player_y >= yFootball - 7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight 
            elif yFootball - 7 <= 21.6 and not(player_x > los - 1.5 and player_y >= 21.6 and player_y <= yFootball + 7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
                
    weight_by_height = 0
    if number_players != 0:
        weight_by_height=value/number_players
    else:
        weight_by_height = 0
            
    return weight_by_height

In [144]:
transf_def["HeightByWeightHook"] = transf_def["id"].apply(calculateHeightByWeightHook)

In [147]:
def calculateHeightByWeightCurl(playId): # está entre la zona hook y los números (12 yd hasta 21.6 yd y 31.7 yd hasta 41.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_curl = -1
    
    if playDirection == "right":
        x_curl = los + 15
    elif playDirection == "left":
        x_curl = los - 15
    
    value = 0
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball - 7 < 21.6 and yFootball + 7 > 31.7 and not((player_x < los +1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x < los + 1.5 and player_y < 21.6 and player_y >= yFootball -7)):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
            elif yFootball + 7 > 31.7 and not(player_x < los + 1.5 and player_y > 31.7 and player_y <= yFootball + 7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
            elif yFootball - 7 < 21.6 and not(player_x < los + 1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
        elif playDirection == "left" and player_x >= x_curl and ((player_y >= 12 and player_y < 21.6) or (player_y > 31.7 and player_y <= 41.3)):
            if yFootball + 7 > 31.7 and yFootball - 7 < 21.6 and not((player_x > los - 1.5 and player_y > 31.7 and player_y <= yFootball + 7) or (player_x > los -1.5 and player_y < 21.6 and player_y >= yFootball - 7)):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
            elif yFootball + 7 > 31.7 and not(player_x > los -1.5 and player_y > 31.7 and player_y <= yFootball +7):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
            elif yFootball - 7 < 21.6 and not(player_x > los -1.5 and player_y >= yFootball - 7 and player_y < 21.6):
                number_players+=1
                player_height = players.loc[row["nflId"]]["height"]
                player_weight = players.loc[row["nflId"]]["weight"]
                value += player_height/player_weight
                
                
    weight_by_height = 0
    if number_players != 0:
        weight_by_height=value/number_players
    else:
        weight_by_height = 0
            
    return weight_by_height

In [148]:
transf_def["HeightByWeightCurl"] = transf_def["id"].apply(calculateHeightByWeightCurl)

In [149]:
def calculateHeightByWeightFlat(playId): # está fuera de los números (0 yd hasta 12 yd y 41.3 yd hasta 53.3 yd)
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    defenders = defenses[defenses["id"]==playId]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    x_flat = -1
    
    if playDirection == "right":
        x_flat = los + 15
    elif playDirection == "left":
        x_flat = los - 15
    
    value = 0
    number_players=0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        player_x = row["x"]
        player_y = row["y"]
        if playDirection == "right" and player_x <= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
            player_height = players.loc[row["nflId"]]["height"]
            player_weight = players.loc[row["nflId"]]["weight"]
            value += player_height/player_weight
        elif playDirection == "left" and player_x >= x_flat and ((player_y > 0 and player_y < 12) or (player_y > 41.3 and player_y < 53.3)):
            number_players+=1
            player_height = players.loc[row["nflId"]]["height"]
            player_weight = players.loc[row["nflId"]]["weight"]
            value += player_height/player_weight
            
    weight_by_height = 0
    if number_players != 0:
        weight_by_height=value/number_players
    else:
        weight_by_height = 0
            
    return weight_by_height

In [150]:
transf_def["HeightByWeightFlat"] = transf_def["id"].apply(calculateHeightByWeightFlat)

In [151]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,3,2,2,left,3,4,3,2,5,True,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,2,2,3,right,3,4,2,3,5,True,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,2,2,3,left,5,2,4,1,5,True,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,3,1,3,right,2,5,2,3,5,True,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,2,3,3,left,3,5,3,2,5,True,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,2,3,2,right,3,4,3,2,5,True,-3.91,0,-2,0.000000,0.017582,0.019208,0.000000
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,2,2,3,right,3,4,3,2,5,True,-0.97,0,-2,0.000000,0.016636,0.018979,0.019819
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,2,1,4,left,3,4,2,3,5,True,0.07,-1,-1,0.018878,0.016636,0.019862,0.019789
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,3,2,2,right,3,4,2,3,5,True,-0.94,-1,-1,0.000000,0.019526,0.019136,0.020001


In [155]:
def calculateWeightByArea(playId,area):
    defenders = defenses[defenses["id"]==playId]
    
    weight = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
        weight += players.loc[row["nflId"]]["weight"]
        
    return weight/area

In [156]:
transf_def["WeightByArea"] = transf_def.apply(lambda x: calculateWeightByArea(x["id"],x["defenseArea"]),axis=1)

In [157]:
def calculateWeightByAreaNoLine(playId,area):
    defenders = defenses[defenses["id"]==playId]
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    
    weight = 0
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        player_x = row["x"]
        player_y = row["y"]
        
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        weight += players.loc[row["nflId"]]["weight"]
        
    return weight/area

In [158]:
transf_def["WeightByArea"] = transf_def.apply(lambda x: calculateWeightByAreaNoLine(x["id"],x["defenseAreaCoverDefenders"]),axis=1)

Densidades

In [159]:
def calculateDensity(playId,area,deep,hook,curl,flat,dl):
    return (deep + hook + curl + flat + dl) / area

In [160]:
transf_def["density"] = transf_def.apply(lambda x: calculateDensity(x["id"],x["defenseArea"],x["deepzonePlayers"],x["hookzonePlayers"],x["curlzonePlayers"],x["flatzonePlayers"],x["defensivelinezonePlayers"]),axis=1)

In [161]:
def calculateDensityNoLine(playId,area,deep,hook,curl,flat):
    return (deep + hook + curl + flat) / area

In [162]:
transf_def["densityNoLine"] = transf_def.apply(lambda x: calculateDensityNoLine(x["id"],x["defenseAreaCoverDefenders"],x["deepzonePlayers"],x["hookzonePlayers"],x["curlzonePlayers"],x["flatzonePlayers"]),axis=1)

In [15]:
def calculateDensityInsidePoints(playId):
    defenders = defenses[defenses["id"]==playId]
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    hull_indices = hull.vertices
    
    polygon = [position_players[e] for i, e in enumerate(hull_indices,0)]
    
    
    return len(defenders) - len(polygon) / hull.volume

In [166]:
transf_def["densityInsidePoints"] = transf_def["id"].apply(calculateDensityInsidePoints)

In [17]:
def calculateDensityInsidePointsNoLine(playId):
    defenders = defenses[defenses["id"]==playId]
    los = plays_no_rz[plays_no_rz["id"]==playId]["absoluteYardlineNumber"].values[0]
    playDirection = defenses[defenses["id"]==playId]["playDirection"].values[0]
    yFootball = all_plays[(all_plays["id"]==playId)&(all_plays["displayName"]=="Football")]["y"].values[0]
    x_defensiveline = -1
    
    if playDirection == "right":
        x_defensiveline = los + 1.5
    elif playDirection == "left":
        x_defensiveline = los - 1.5
    position_players = []
    
    for index,row in defenders.iterrows():
        if row["position"] in ["DL","NT","DT","DE"]:
            continue
            
        player_x = row["x"]
        player_y = row["y"]
        
        if playDirection == "right" and player_x <= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue
        elif playDirection == "left" and player_x >= x_defensiveline and (player_y >= yFootball -7 and player_y <= yFootball + 7):
            continue

        position_players.append((row["x"],row["y"]))
    
    hull = ConvexHull(position_players)
    hull_indices = hull.vertices
    
    polygon = [position_players[e] for i, e in enumerate(hull_indices,0)]
    
    return len(defenders) - len(polygon) / hull.volume

In [169]:
transf_def["densityInsidePointsNoLine"] = transf_def["id"].apply(calculateDensityInsidePointsNoLine)

In [48]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine,numberOffensivePlayersAnotherPosition
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,3,2,2,left,3,4,3,2,5,True,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034,3.376032,0.036025,0.036025,6.974268,6.974268,0
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,2,2,3,right,3,4,2,3,5,True,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455,3.178611,0.034284,0.034284,6.975511,6.975511,0
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,2,2,3,left,5,2,4,1,5,True,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745,2.337434,0.025211,0.025211,6.985594,6.985594,0
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,3,1,3,right,2,5,2,3,5,True,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455,3.192871,0.037004,0.036351,6.973569,6.969707,0
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,2,3,3,left,3,5,3,2,5,True,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279,4.281979,0.042752,0.045692,7.978624,7.980418,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,2,3,2,right,3,4,3,2,5,True,-3.91,0,-2,0.000000,0.017582,0.019208,0.000000,4.867507,0.041344,0.048035,6.970468,6.959971,0
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,2,2,3,right,3,4,3,2,5,True,-0.97,0,-2,0.000000,0.016636,0.018979,0.019819,2.120069,0.021384,0.021384,6.987781,6.987781,0
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,2,1,4,left,3,4,2,3,5,True,0.07,-1,-1,0.018878,0.016636,0.019862,0.019789,2.307450,0.023858,0.023858,6.982958,6.982958,0
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,3,2,2,right,3,4,2,3,5,True,-0.94,-1,-1,0.000000,0.019526,0.019136,0.020001,2.948817,0.031275,0.031275,6.982128,6.982128,0


In [44]:
#save dataset
pickle_file = open('../resources/processing_defenses/transf_def.pickle','wb')
pickle.dump(transf_def,pickle_file)
pickle_file.close()

In [9]:
#load dataset
pickle_file = open('../resources/processing_defenses/transf_def.pickle','rb')
transf_def = pickle.load(pickle_file)
pickle_file.close()

In [25]:
transf_def

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberOffensivePlayersAnotherPosition,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine
0,2018090600:75:1,0,1,3,2,1,7,True,194.31095,194.31095,22.96,15.25,left,1,2,1,1,1,0,3,2,2,left,3,4,3,2,5,True,-0.63,0,-2,0.020227,0.018643,0.020066,0.021034,3.376032,0.036025,0.036025,6.974268,6.974268
1,2018090600:146:1,0,1,3,2,1,7,True,204.17725,204.17725,29.22,12.57,left,1,3,1,1,0,0,2,2,3,right,3,4,2,3,5,True,-1.15,-1,-1,0.020227,0.019228,0.020356,0.020455,3.178611,0.034284,0.034284,6.975511,6.975511
2,2018090600:168:1,0,1,2,2,2,7,True,277.65495,277.65495,33.67,15.21,left,1,2,1,2,0,0,2,2,3,left,5,2,4,1,5,True,0.71,-1,-1,0.020227,0.017683,0.020997,0.020745,2.337434,0.025211,0.025211,6.985594,6.985594
3,2018090600:190:1,1,0,2,3,1,7,True,189.16960,165.05520,26.42,12.91,left,1,3,1,1,0,0,3,1,3,right,2,5,2,3,5,True,-3.11,0,-2,0.000000,0.020120,0.021193,0.020455,3.192871,0.037004,0.036351,6.973569,6.969707
5,2018090600:320:1,1,0,3,3,1,8,True,187.12510,153.20020,27.80,13.29,right,1,3,1,1,0,0,2,3,3,left,3,5,3,2,5,True,-4.82,0,-3,0.000000,0.018995,0.019537,0.021279,4.281979,0.042752,0.045692,7.978624,7.980418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19112,2018123015:3819:17,1,0,3,3,0,7,True,169.30945,124.90995,23.43,12.71,left,1,2,2,1,0,0,2,3,2,right,3,4,3,2,5,True,-3.91,0,-2,0.000000,0.017582,0.019208,0.000000,4.867507,0.041344,0.048035,6.970468,6.959971
19113,2018123015:3932:17,0,0,1,4,2,7,True,327.34785,327.34785,35.76,13.20,left,1,3,1,1,0,0,2,2,3,right,3,4,3,2,5,True,-0.97,0,-2,0.000000,0.016636,0.018979,0.019819,2.120069,0.021384,0.021384,6.987781,6.987781
19114,2018123015:3969:17,0,1,1,4,1,7,True,293.39745,293.39745,26.24,14.67,left,1,3,1,1,0,0,2,1,4,left,3,4,2,3,5,True,0.07,-1,-1,0.018878,0.016636,0.019862,0.019789,2.307450,0.023858,0.023858,6.982958,6.982958
19115,2018123015:4057:17,0,0,2,3,2,7,True,223.81860,223.81860,36.23,11.62,right,1,3,1,1,0,0,3,2,2,right,3,4,2,3,5,True,-0.94,-1,-1,0.000000,0.019526,0.019136,0.020001,2.948817,0.031275,0.031275,6.982128,6.982128


In [25]:
transf_def.drop(["checkNumberIsGood","numberCoverPlayers","numberPlayersOffense","attackerIsGood","playDirection"],axis=1,inplace=True)

In [31]:
len(transf_def)

16498

In [24]:
transf_def[transf_def["numberCoverPlayers"] != transf_def["numberPlayersDefenseStrongSide"] + transf_def["numberPlayersDefenseWeakSide"]]

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberOffensivePlayersAnotherPosition,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine


In [29]:
transf_def[transf_def["numberQBs"]+transf_def["numberWRs"]+transf_def["numberTEs"]+transf_def["numberRBs"]+transf_def["numberFBs"]+transf_def["numberOffensivePlayersAnotherPosition"] -1!= transf_def["numberPlayersOffenseStrongSide"] + transf_def["numberPlayersOffenseWeakSide"]]

,id,defensivelinezonePlayers,deepzonePlayers,hookzonePlayers,curlzonePlayers,flatzonePlayers,numberCoverPlayers,checkNumberIsGood,defenseArea,defenseAreaCoverDefenders,width,height,playDirection,numberQBs,numberWRs,numberTEs,numberRBs,numberFBs,numberOffensivePlayersAnotherPosition,numberSafeties,numberLBs,numberCBs,strongSide,numberPlayersDefenseStrongSide,numberPlayersDefenseWeakSide,numberPlayersOffenseStrongSide,numberPlayersOffenseWeakSide,numberPlayersOffense,attackerIsGood,differenceOffenseVsDefenseWidth,differenceOffenseVsDefenseStrongSide,differenceOffenseVsDefenseWeakSide,HeightByWeightDeep,HeightByWeightHook,HeightByWeightCurl,HeightByWeightFlat,WeightByArea,density,densityNoLine,densityInsidePoints,densityInsidePointsNoLine


In [30]:
transf_def.to_csv("../processed_data/transformed_data/static_defenses.csv")